In [155]:
import json
import math
from collections import defaultdict
import pprint
import re
import fnmatch
import requests
import base64
import pandas as pd


#Team Selection

Team = str(input("Select team to see stats: ")).title().replace('And','and').replace('&','and')
print("Selected Team is:",Team)
Team_2_Names = Team.split(" ")


#Opening JSON file
try:
    url = 'https://raw.githubusercontent.com/Giray18/Kaggle_exercises/main/epl_2022_2023_07_02_2023.json'
    page = requests.get(url)
    obj = json.loads((page.text).replace('&','and'))       
except UnicodeDecodeError:
    print("Content could not be read could be UTF-8 non compliance")
else:
    #Getting game ids
    Game_ids = []
    for key, value in obj.items(): 
        Game_ids.append(key)

    def Total_Goals(Team): #Calculating Total Team Goals
        Team_Scores = [obj[str(i)]["event"][0] for i in Game_ids if (Team) in obj[str(i)]["event"][0]] #Getting Team Match Results
        Stripped = [item.split(" ") for item in Team_Scores]
        Goals = 0
        for i in range(len(Stripped)): 
            if len(Team_2_Names) > 1 or Team in Team_Scores:
                position = (Stripped[i].index((Team_2_Names[0])))
                Goals += int((Stripped[i][position + len(Team_2_Names)]).replace(",","").replace(".",""))
            else:
                position = (Stripped[i].index((Team)))
                Goals += int((Stripped[i][position + 1]).replace(",","").replace(".",""))
        return Goals
   
    def Total_Shots(Team): #Calculating Total Team Shoots
        Shoots = 0
        for i in Game_ids:
            if (Team) == obj[str(i)]["team2_name"]:
                Shoots += int((obj[str(i)]['team2_stat']['shots']))
            elif (Team) == obj[str(i)]["team1_name"]:
                Shoots = Shoots + int((obj[str(i)]["team1_stat"]["shots"]))
        return Shoots

       
    def Shot_Goal_Percentage(Total_Goals,Total_Shots,Team): #Calculating Successful Shots
        Total_Goals = int(Total_Goals(Team))
        Total_Shoots = int(Total_Shots(Team))
        percent = Total_Goals/Total_Shoots
        percentage = "{:.2%}".format(percent)
        return percentage

    #Team_Scores = [obj[str(i)]["event"] for i in Game_ids if (Team) in obj[str(i)]["event"]] #Getting Team Match Results
    
    #print(obj)
    # Team = [obj[str(i)]["team1_name"] for i in Game_ids]
    # Team.extend(obj[str(i)]["team2_name"] for i in Game_ids)
    # #print(len(Team))
    # Unique_team = pd.Series(Team).drop_duplicates().tolist()
    #print(Unique_team)


    # Players_startings = [obj[str(i)]["team1_startings"] for i in Game_ids]
    # Players_startings.extend(obj[str(i)]["team2_startings"] for i in Game_ids)
    # Player_startings_2 = []
    # for i in Players_startings:
    #     for a in i:
    #         Player_startings_2.append(a)
    # Unique_players = pd.Series(Player_startings_2).drop_duplicates()

    # dicts_teams = {}
    # for i in Unique_team:
    #     dicts_teams[i] = [Team.count(i)]
    # print(dicts_teams)


    # dicts_players = {}
    # for i in Unique_players:
    #     dicts_players[i] = [Player_startings_2.count(i)]
    # sorted_footballers_by_goals = sorted(dicts_players.items(), key=lambda x:x[1], reverse = True)[:10]
    # df = pd.DataFrame(data=sorted_footballers_by_goals,columns=['Player_Name','Games_Started'])
    # print(df)



    def Most_10_starting_players(obj): #List of 10 players has most starting lineup
        Players_startings = [obj[str(i)]["team1_startings"] for i in Game_ids]
        Players_startings.extend(obj[str(i)]["team2_startings"] for i in Game_ids)
        Player_startings_2 = []
        for i in Players_startings:
            for a in i:
                Player_startings_2.append(a)
        Unique_players = pd.Series(Player_startings_2).drop_duplicates()
        dicts_players = {}
        for i in Unique_players:
            dicts_players[i] = [Player_startings_2.count(i)]
        sorted_footballers_by_goals = sorted(dicts_players.items(), key=lambda x:x[1], reverse = True)[:10]
        df = pd.DataFrame(data=sorted_footballers_by_goals,columns=['Player_Name','Games_Started'])
        return df


    def top_10_scorers_of_team(Team): #create list of top 10 goal scorers of requested team
        Goal_scorers = []
        Goal_scorers_2 = []
        Team_Scores = []
        for i in Game_ids:
            if (Team) in obj[str(i)]["event"][0]:
                Team_Scores.append(obj[str(i)]["event"][:])
        for i in Team_Scores:
            for a in i:
                if 'Goal!' in a:
                    ab = re.split('[0-9].',a)[2].split(" ")[1:3]
                    Goal_scorers.append(ab)
        for i in Goal_scorers:
            Goal_scorers_2.append(str(i).replace(",",""))
        Unique_players = pd.Series(Goal_scorers_2).drop_duplicates()
        dicts_players = {}
        for i in Unique_players:
            dicts_players[i] = [Goal_scorers_2.count(i)]
        sorted_footballers_by_goals = sorted(dicts_players.items(), key=lambda x:x[1], reverse = True)[:10]
        df = pd.DataFrame(data=sorted_footballers_by_goals,columns=['Player_Name','Goals'])
        return df

print(Most_10_starting_players(obj))
print("Top 10 scorers:",top_10_scorers_of_team(Team))
print("Total Shots:",Total_Shots(Team))
print("Total Goals:",Total_Goals(Team))
print("Shot to Goal Percentage:",Shot_Goal_Percentage(Total_Goals,Total_Shots,Team))


Selected Team is: Manchester City
         Player_Name Games_Started
0         Harry Kane          [22]
1           Tim Ream          [22]
2    Andreas Pereira          [22]
3    James Tarkowski          [21]
4  Vitaliy Mykolenko          [21]
5         Alex Iwobi          [21]
6         Danny Ward          [21]
7       David de Gea          [21]
8         David Raya          [21]
9     Mathias Jensen          [21]
Top 10 scorers:              Player_Name Goals
0   ['Erling' 'Haaland']  [25]
1       ['Phil' 'Foden']   [7]
2   ['Julián' 'Álvarez']   [5]
3     ['Riyad' 'Mahrez']   [4]
4         ['Kevin' 'De']   [3]
5       ['Ivan' 'Toney']   [2]
6     ['João' 'Cancelo']   [2]
7  ['Anthony' 'Martial']   [2]
8    ['Jack' 'Grealish']   [2]
9   ['Bernardo' 'Silva']   [2]
Total Shots: 3484
Total Goals: 54
Shot to Goal Percentage: 1.55%
